In [31]:
import boto3
import os
import csv
from dotenv import load_dotenv

In [32]:
## load AWS credentials from env file
load_dotenv()
s3 = boto3.resource('s3',
    aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
)

In [33]:
bucket_name = 'yaow-for-cloud-infra'
try:
    s3.create_bucket(Bucket = bucket_name, CreateBucketConfiguration = {
        'LocationConstraint': 'us-west-2'
    })
except Exception as e:
    print(e)

In [34]:
bucket = s3.Bucket(bucket_name)
bucket.Acl().put(ACL = 'public-read')

{'ResponseMetadata': {'RequestId': '58S2YRFBJM87YXWN',
  'HostId': 't7vlwtEXVuRyLpOXQvl467n90G7aZ84v1bH+grto8iFBFMQ2wybkpowlFues2XN+rwBw23ccwxU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 't7vlwtEXVuRyLpOXQvl467n90G7aZ84v1bH+grto8iFBFMQ2wybkpowlFues2XN+rwBw23ccwxU=',
   'x-amz-request-id': '58S2YRFBJM87YXWN',
   'date': 'Tue, 19 Oct 2021 06:50:23 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1}}

In [35]:
dyndb = boto3.resource('dynamodb',
    region_name = 'us-west-2',
    aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
)

In [78]:
try:
    table = dyndb.create_table(
        TableName = 'DataTable',
        KeySchema = [
            {
                'AttributeName': 'Id',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'Temp',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions = [
            {
                'AttributeName': 'Id',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'Temp',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput = {
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
except Exception as e:
    print(e)

In [79]:
print("Table status:", table.table_status)

Table status: CREATING


In [80]:
with open('experiments.csv', 'r') as csvfile:
    csvf = csv.reader(csvfile, delimiter = ',', quotechar = '|')
    is_header = False
    for item in csvf:
        if is_header == False:
            is_header = True
            continue
        body = open(item[4], 'rb')
        s3.Object(bucket_name, item[4]).put(Body = body)
        md = s3.Object(bucket_name, item[4]).Acl().put(ACL='public-read')

        url = 'https://s3-us-west-2.amazonaws.com/' + bucket_name + '/' + item[4]
        metadata_item = {'Id': item[0], 'Temp': item[1], 
                'Conductivity': item[2], 'Concentration': item[3], 'url': url}
        try:
            table.put_item(Item = metadata_item)
        except Exception as e:
            print("item may already be there or another failure " + e)

{'Id': '1', 'Temp': '-1', 'Conductivity': '52', 'Concentration': '3.4', 'url': 'https://s3-us-west-2.amazonaws.com/yaow-for-cloud-infra/exp1.csv'}
{'Id': '2', 'Temp': '-2', 'Conductivity': '52.1', 'Concentration': '3.4', 'url': 'https://s3-us-west-2.amazonaws.com/yaow-for-cloud-infra/exp2.csv'}
{'Id': '3', 'Temp': '-2.93', 'Conductivity': '57.1', 'Concentration': '3.7', 'url': 'https://s3-us-west-2.amazonaws.com/yaow-for-cloud-infra/exp3.csv'}


In [82]:
response = table.get_item(
    Key = {
        'Id': '1',
        'Temp': '-1'
    }
)
item = response['Item']
print(item)

{'Concentration': '3.4', 'Temp': '-1', 'Id': '1', 'url': 'https://s3-us-west-2.amazonaws.com/yaow-for-cloud-infra/exp1.csv', 'Conductivity': '52'}


In [83]:
response

{'Item': {'Concentration': '3.4',
  'Temp': '-1',
  'Id': '1',
  'url': 'https://s3-us-west-2.amazonaws.com/yaow-for-cloud-infra/exp1.csv',
  'Conductivity': '52'},
 'ResponseMetadata': {'RequestId': '5MDFQH05KP4SR8QS3LJCQ9KSH7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 19 Oct 2021 21:42:40 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '176',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5MDFQH05KP4SR8QS3LJCQ9KSH7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1584997855'},
  'RetryAttempts': 0}}